In [1]:
!BEGIN 6_Interleaved_Vertex_Data

# 交错顶点数据

每个顶点是一个 具有 位置颜色等信息的结构体 组合成一个结构体数组

* 创建交错顶点数据 结构体
* 控制着色器的颜色属性

In [2]:
%%file ../code/6_Interleaved_Vertex_Data/Vertex.h
#pragma once
#include <GL/glew.h>
#include <cstddef>

struct Vertex {
    // 保持字节数是 4 的倍数 否则进行填充对齐
    struct Position {
        float x;
        float y;
    } position;
    struct Color {
        GLubyte r;
        GLubyte g;
        GLubyte b;
        GLubyte a;
    } color;
};

Overwriting ../code/6_Interleaved_Vertex_Data/Vertex.h


## 在 Sprite::init 和 Sprite::draw 中 更改 顶点类型  

In [3]:
%%file ../code/6_Interleaved_Vertex_Data/Sprite.cpp
#include "Sprite.h"
#include "Vertex.h"
    
Sprite::Sprite() {
    _vboID = 0;
}

Sprite::~Sprite() {
    // 删除缓冲区
    if (_vboID != 0) {
        // args: buffer数量; id的指针 
        glDeleteBuffers(1, &_vboID);
    }
}

void Sprite::init(float x, float y, float width, float height) {
    _x = x;
    _y = y;
    _width = width;
    _height = height;

    // 没有创建缓冲区 则创建
    if (_vboID == 0) {
        // args: 创建 buffer 数; 保存 id 的 指针; 
        glGenBuffers(1, &_vboID);
    }

    // 保存 6 个顶点 精灵是 正方形 由两个三角形组成
    Vertex vertexData[6];
    // 坐标系 设 左下为原点
    // 三角形1
    // 右上
    vertexData[0].position.x = x + width;
    vertexData[0].position.y = y + height;
    // 左上
    vertexData[1].position.x = x;
    vertexData[1].position.y = y + height;
    // 左下
    vertexData[2].position.x = x;
    vertexData[2].position.y = y;
    // 三角形2
    // 右下
    vertexData[3].position.x = x + width;
    vertexData[3].position.y = y;
    // 右上
    vertexData[4].position.x = x + width;
    vertexData[4].position.y = y + height;
    // 左下
    vertexData[5].position.x = x;
    vertexData[5].position.y = y;

    // 紫色
    for (int i = 0; i < 6; i++) {
        vertexData[i].color.r = 255;
        vertexData[i].color.g = 0;
        vertexData[i].color.b = 255;
        vertexData[i].color.a = 255;
    }
    // 蓝色 左上 
    vertexData[1].color = {0, 0, 255, 255};
    // 绿色 右下
    vertexData[3].color = {0, 255, 0, 255};
    
    
    // 告知 GL 哪个 buffer 激活
    // args: buffer的类型; ID
    glBindBuffer(GL_ARRAY_BUFFER, _vboID);

    // 向缓冲区发送数据
    // args: buffer的类型; 字节数; buffer 开头的指针; 用法(指示绘制一次还是多次)
    glBufferData(GL_ARRAY_BUFFER, sizeof(vertexData), vertexData, GL_STATIC_DRAW);

    // 解绑缓冲区
    glBindBuffer(GL_ARRAY_BUFFER, 0);
}

void Sprite::draw() {
    glBindBuffer(GL_ARRAY_BUFFER, _vboID);

    // 告诉 OpenGL 从缓冲区的哪个位置绘制 
    // args: 使用的 属性索引 ; 每个顶点的该属性读取数量; 类型; 
    //       规范化: 转换为 -1 - 1 的数; 
    //       步幅; 属性所在偏移量 (void *)
    // 这里的2 后续可以 通过 in vec2 读取到  2 个 float
    // 由于位置在 Vertex 第一个位置存储 偏移量设置为0
    glVertexAttribPointer(0, 2, GL_FLOAT, GL_FALSE, 
                          sizeof(Vertex), (void *) offsetof(Vertex, position));
    glVertexAttribPointer(1, 4, GL_UNSIGNED_BYTE, GL_TRUE, 
                          sizeof(Vertex), (void *) offsetof(Vertex, color));

    glEnableVertexAttribArray(0);
    glEnableVertexAttribArray(1);
    // 绘制 
    // args: 绘制的类型; 开始索引; 顶点个数
    glDrawArrays(GL_TRIANGLES, 0, 6);
    
    glDisableVertexAttribArray(0);
    glDisableVertexAttribArray(1);

    glBindBuffer(GL_ARRAY_BUFFER, 0);
}

Overwriting ../code/6_Interleaved_Vertex_Data/Sprite.cpp


## MainGame::initShader 中 添加 属性 

In [4]:
%%file ../code/6_Interleaved_Vertex_Data/MainGame.cpp
#include "MainGame.h"
#include "Sprite.h"    
#include "Errors.h"
#include <iostream>
#include <string>

MainGame::MainGame() { 
    _window = nullptr;
    _screenWidth = 1024;
    _screenHeight = 768;
    _gameState = GameState::PLAY;
}

MainGame::~MainGame() {
}

void MainGame::run() {
    initSystems();

    // 从左下到右上
    _sprite.init(-1.0f, -1.0f, 2.0f, 2.0f);
    
    gameLoop();
}

// 初始化各种系统
void MainGame::initSystems() {
    // 初始化SDL
    SDL_Init(SDL_INIT_EVERYTHING);
    // 创建窗口
    _window = SDL_CreateWindow("Game Engine", SDL_WINDOWPOS_CENTERED, 
                                SDL_WINDOWPOS_CENTERED, _screenWidth, _screenHeight,
                                SDL_WINDOW_OPENGL);
    if (_window == nullptr) {
        fatalError("SDL Window could not be created!");
    }
    // 初始化 GL 上下文
    SDL_GLContext glContex = SDL_GL_CreateContext(_window);
    if (glContex == nullptr) {
        fatalError("SDL_GL context could not be created!");
    }
    // 初始化 glew
    GLenum error = glewInit();
    if (error != GLEW_OK) {
        fatalError("Could not initalize glew!");
    }
    // 启用双缓冲区
    SDL_GL_SetAttribute(SDL_GL_DOUBLEBUFFER, 1);
    // 设置背景
    glClearColor(0.0f, 0.0f, 1.0f, 1.0f);

    initShaders();
}

void MainGame::initShaders() {
    _colorProgram.compileShaders("Shaders/colorShading.vert", 
                                 "Shaders/colorShading.frag");
    _colorProgram.addAttribute("vertexPosition");
    
    _colorProgram.addAttribute("vertexColor");
    _colorProgram.linkShaders();
}

// 处理输入
void MainGame::processInput() {
    SDL_Event evnt;
    while(SDL_PollEvent(&evnt)) {
        switch (evnt.type) {
            case SDL_QUIT:
                _gameState = GameState::EXIT;
                break;
            case SDL_MOUSEMOTION:
                // std::cout << evnt.motion.x << " " << evnt.motion.y << std::endl;
                break;
        }
    }
}
    
void MainGame::gameLoop() {
    while (_gameState != GameState::EXIT) {
        processInput();
        drawGame();
    }
}

// 绘制游戏画面
void MainGame::drawGame() {
    glClearDepth(1.0); 
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT); 

    _colorProgram.use();

    _sprite.draw();
    
    _colorProgram.unuse();

    SDL_GL_SwapWindow(_window);
}

Overwriting ../code/6_Interleaved_Vertex_Data/MainGame.cpp


## GLSL 代码中 更改为依据传递的顶点颜色绘制

In [5]:
%%file ../code/6_Interleaved_Vertex_Data/Shaders/colorShading.vert
#version 130

in vec2 vertexPosition;
// OpenGL 会将 传入 byte 转 float
in vec4 vertexColor;

// 确保两个阶段使用相同名称 可以进行传递
out vec4 fragmentColor;

void main() {
    gl_Position.xy = vertexPosition;
    // 深度 
    gl_Position.z = 0.0;
    // 归一化
    gl_Position.w = 1.0;
    fragmentColor = vertexColor;
}

Overwriting ../code/6_Interleaved_Vertex_Data/Shaders/colorShading.vert


In [6]:
%%file ../code/6_Interleaved_Vertex_Data/Shaders/colorShading.frag
#version 130

// flat 可以 取消差值渐变 忽略一些点
in vec4 fragmentColor;
// 片段着色器文件

out vec4 color;

void main() {
    color = fragmentColor;
}   

Overwriting ../code/6_Interleaved_Vertex_Data/Shaders/colorShading.frag


In [7]:
!END 6_Interleaved_Vertex_Data

Building and running 6_Interleaved_Vertex_Data...
